In [33]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from PIL import Image

In [34]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=78):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [35]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [36]:
dataset_path = "./Train"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [37]:
train_size = int(0.85 * len(dataset))
val_size = int(0.075 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [38]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel(num_classes=len(dataset.classes)).to(device)

In [40]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [41]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step()
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

Epoch 1/10, Loss: 1700.0440, Accuracy: 50.68%
Epoch 2/10, Loss: 743.7489, Accuracy: 76.02%
Epoch 3/10, Loss: 578.7818, Accuracy: 80.97%
Epoch 4/10, Loss: 496.9485, Accuracy: 83.62%
Epoch 5/10, Loss: 455.7138, Accuracy: 84.95%
Epoch 6/10, Loss: 425.9998, Accuracy: 85.70%
Epoch 7/10, Loss: 402.9836, Accuracy: 86.65%
Epoch 8/10, Loss: 377.7350, Accuracy: 87.32%
Epoch 9/10, Loss: 364.0753, Accuracy: 87.73%
Epoch 10/10, Loss: 355.4467, Accuracy: 88.12%


In [42]:
torch.save(model.state_dict(), "armenian_model.pth")

In [43]:
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 88.16%


In [56]:
def test_single_image(image_path, model, device):
    transform = transforms.Compose([
        transforms.Grayscale(),  
        transforms.Resize((64, 64)),  
        transforms.ToTensor(), 
        transforms.Normalize((0.5,), (0.5,))  
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

    image = image.to(device)
    model = model.to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)

    armenian_dict = {
        0: "Ա", 1: "Բ", 2: "Գ", 3: "Դ", 4: "Ե", 5: "Զ", 6: "Է", 7: "Ը", 8: "Թ", 9: "Ժ",
        10: "Ի", 11: "Լ", 12: "Խ", 13: "Ծ", 14: "Կ", 15: "Հ", 16: "Ձ", 17: "Ղ", 18: "Ճ",
        19: "Մ", 20: "Յ", 21: "Ն", 22: "Շ", 23: "Ո", 24: "Ու", 25: "Չ", 26: "Պ", 27: "Ջ",
        28: "Ռ", 29: "Ս", 30: "Վ", 31: "Տ", 32: "Ր", 33: "Ց", 34: "Փ", 35: "Ք", 36: "Եվ",
        37: "Օ", 38: "Ֆ", 39: "ա", 40: "բ", 41: "գ", 42: "դ", 43: "ե", 44: "զ", 45: "է",
        46: "ը", 47: "թ", 48: "ժ", 49: "ի", 50: "լ", 51: "խ", 52: "ծ", 53: "կ", 54: "հ",
        55: "ձ", 56: "ղ", 57: "ճ", 58: "մ", 59: "յ", 60: "ն", 61: "շ", 62: "ո", 63: "ու",
        64: "չ", 65: "պ", 66: "ջ", 67: "ռ", 68: "ս", 69: "վ", 70: "տ", 71: "ր", 72: "ց",
        73: "փ", 74: "ք", 75: "և", 76: "օ", 77: "ֆ"
    }

    predicted_label = armenian_dict[predicted_class.item()]
    print(f"Predicted class: {predicted_class.item()}")
    print(f"Predicted Armenian letter: {predicted_label}")


image_path = "3.jpeg"
test_single_image(image_path, model, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Predicted class: 1
Predicted Armenian letter: Բ
